In [0]:
# ===========================================
# 1) Ler a tabela gold
# ===========================================
df = spark.table("classes.gold.fato_consumo_agua")

# ===========================================
# 2) Criar o TOP 10 das regiões com maior valor_total
# ===========================================
from pyspark.sql import functions as F

df_top10 = (
    df.groupBy("regiao")
      .agg(F.max("valor_total").alias("maior_valor"))
      .orderBy(F.desc("maior_valor"))
      .limit(10)
)

display(df_top10)

# ===========================================
# 3) GRÁFICO DE COLUNA (BARRAS VERTICAIS)
# ===========================================
p = df_top10.plot(
    x="regiao",
    y="maior_valor",
    kind="bar",
    title="TOP 10 • Regiões com Maior valor_total",
)

display(p)

In [0]:
# ==============================
# Scatter plot: Maior valor por ano
# ==============================
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# 1) Ler a tabela GOLD
df = spark.table("classes.gold.fato_consumo_agua")

# 2) Garantir colunas necessárias existem
# (Se sua tabela já tem 'ano' como inteiro e 'valor_total' numeric, os casts abaixo são seguros)
df = df.withColumn("ano", F.col("ano").cast("int")) \
       .withColumn("valor_total", F.col("valor_total").cast("double"))

# 3) Janela para pegar o maior valor por ano
w = Window.partitionBy("ano").orderBy(F.desc("valor_total"))

df_maior_ano = (
    df.withColumn("rk", F.row_number().over(w))
      .filter(F.col("rk") == 1)   # pega apenas o TOP 1 de cada ano
      .select("ano", "regiao", "valor_total")
      .orderBy("ano")
)

display(df_maior_ano)   # opcional: ver os dados antes do gráfico

# 4) Plot de bolinhas (scatter)
# x = ano (numérico), y = valor_total (numérico)
p = df_maior_ano.plot(
    kind="scatter",
    x="ano",
    y="valor_total",
    title="Scatter • Maior Valor Total por Ano",
)

display(p)

In [0]:
from pyspark.sql import functions as F

# ==============================
# 1. Ler a tabela GOLD já formatada
# ==============================
df = spark.table("classes.gold.top10_regioes_maior_consumo_m3_FORMATADO")

# IMPORTANTE:
# A coluna formatada virou string (por causa das vírgulas)
# Vamos criar uma coluna numérica limpa para o gráfico.
df_num = df.withColumn(
    "maior_consumo_m3_num",
    F.regexp_replace("maior_consumo_m3", "\\.", "")  # remove milhares
).withColumn(
    "maior_consumo_m3_num",
    F.regexp_replace("maior_consumo_m3_num", ",", ".").cast("double")
)

display(df_num)

# ==============================
# 2. Gráfico de barras horizontais (barh)
# ==============================
p = df_num.plot(
    kind="barh",
    x="regiao",
    y="maior_consumo_m3_num",
    title="Top 10 Regiões – Maior Consumo por m³ (Barra Horizontal)"
)

display(p)